In [1]:
from hwr.data.generator import IAMSequence
from hwr.constants import ON, SPLIT
from hwr.models.ONNET import ONNET
from matplotlib import pyplot as plt
import numpy as np

In [2]:
model = ONNET(preload=False)

In [3]:
train_seq = IAMSequence(SPLIT.TRAIN, batch_size=20, preprocess=6,
                                   npz=True, inout_ratio=4)
val1_seq = IAMSequence(SPLIT.VAL1, batch_size=100, preprocess=6,
                                  npz=True, inout_ratio=4)
eval_seq = IAMSequence(SPLIT.TEST, batch_size=100, preprocess=6,
                      npz=True, pred=True, pad_to=(900, 225))

In [8]:
model.train(train_seq, val1_seq, epochs=1000, earlystop=5)

Epoch 1/1000
268/269 [============================>.] - ETA: 0s - loss: 7.2725
Epoch 00001: val_loss improved from inf to 14.83046, saving model to /home/jasper/Desktop/fyp/HWR/hwr/../models/iamon/checkpoint/ONNET/2019-09-13-09:45:41/weights.h5
269/269 [==============================] - 19s 72ms/step - loss: 7.2809 - val_loss: 14.8305
Epoch 2/1000
268/269 [============================>.] - ETA: 0s - loss: 7.2267
Epoch 00002: val_loss did not improve from 14.83046
269/269 [==============================] - 17s 64ms/step - loss: 7.2338 - val_loss: 14.8382
Epoch 3/1000
268/269 [============================>.] - ETA: 0s - loss: 7.4138
Epoch 00003: val_loss did not improve from 14.83046
269/269 [==============================] - 17s 65ms/step - loss: 7.4181 - val_loss: 14.8583
Epoch 4/1000
268/269 [============================>.] - ETA: 0s - loss: 7.1768
Epoch 00004: val_loss did not improve from 14.83046
269/269 [==============================] - 17s 64ms/step - loss: 7.1726 - val_loss: 14

KeyboardInterrupt: 

In [ ]:
# model.load_weights("../../models/iamon/ONNET/2019-09-13-09:42:27-2/weights.h5", full_path=True)